In [1]:
# check access to S3 bucket, if empty, need to remount with:
# `sudo s3fs cs205-youtube-data -o use_cache=/tmp -o allow_other -o uid=1001 -o mp_umask=002 -o multireq_max=5 mys3bucket`
!ls mys3bucket/

yt8pm  yt8pm_100th_shard


In [2]:
from pyspark import SparkConf, SparkContext
from pyspark.sql.types import *
import pyspark

In [3]:
conf = SparkConf().setMaster('local').setAppName('TFrecords_loading').set("spark.jars", "ecosystem/spark/spark-tensorflow-connector/target/spark-tensorflow-connector_2.11-1.10.0.jar")
sc = SparkContext(conf = conf)

spark = pyspark.sql.SparkSession(sc)

In [4]:
'''
First, we need to install yum (make sure you are in correct environment!):
sudo apt install yum

Then,

Install Maven:
Icarus answered a very similar question for me. Its not using "yum", but should still work for your purposes. Try,

wget http://mirrors.advancedhosters.com/apache/maven/maven-3/3.6.1/binaries/apache-maven-3.6.1-bin.tar.gz
basically just go to the maven site. Find the version of maven you want. The file type and use the mirror for the wget statement above.

Afterwards the process is easy

Run the wget command from the dir you want to extract maven too.
run the following to extract the tar,

tar xvf apache-maven-3.0.5-bin.tar.gz
move maven to /usr/local/apache-maven

mv apache-maven-3.0.5  /usr/local/apache-maven
Next add the env variables to your ~/.bashrc file

## note - the following commands need to be re-executed when re-starting the instance.

export M2_HOME=/usr/local/apache-maven
export M2=$M2_HOME/bin 
export PATH=$M2:$PATH
Execute these commands

source ~/.bashrc

THEN
git clone the following repo
git clone https://github.com/tensorflow/ecosystem.git

cd ecosystem/hadoop
mvn clean package

then
cd ../../hadoop #or make sure we are in the hadoop directory
mvn clean install

# Build Spark TensorFlow connector
cd ../spark/spark-tensorflow-connector
mvn clean install

and run spark with spark-tensorflow connector!
pyspark --jars ecosystem/spark/spark-tensorflow-connector/target/spark-tensorflow-connector_2.11-1.10.0.jar
'''

'\nFirst, we need to install yum (make sure you are in correct environment!):\nsudo apt install yum\n\nThen,\n\nInstall Maven:\nIcarus answered a very similar question for me. Its not using "yum", but should still work for your purposes. Try,\n\nwget http://mirrors.advancedhosters.com/apache/maven/maven-3/3.6.1/binaries/apache-maven-3.6.1-bin.tar.gz\nbasically just go to the maven site. Find the version of maven you want. The file type and use the mirror for the wget statement above.\n\nAfterwards the process is easy\n\nRun the wget command from the dir you want to extract maven too.\nrun the following to extract the tar,\n\ntar xvf apache-maven-3.0.5-bin.tar.gz\nmove maven to /usr/local/apache-maven\n\nmv apache-maven-3.0.5  /usr/local/apache-maven\nNext add the env variables to your ~/.bashrc file\n\n## note - the following commands need to be re-executed when re-starting the instance.\n\nexport M2_HOME=/usr/local/apache-maven\nexport M2=$M2_HOME/bin \nexport PATH=$M2:$PATH\nExecute 

In [7]:
!rm -r test-output.tfrecord

In [8]:
# the example below is taken from:

# https://github.com/tensorflow/ecosystem/tree/master/spark/spark-tensorflow-connector#usage-examples

path = "test-output.tfrecord"

fields = [StructField("id", IntegerType()), StructField("IntegerCol", IntegerType()),
          StructField("LongCol", LongType()), StructField("FloatCol", FloatType()),
          StructField("DoubleCol", DoubleType()), StructField("VectorCol", ArrayType(DoubleType(), True)),
          StructField("StringCol", StringType())]
schema = StructType(fields)
test_rows = [[11, 1, 23, 10.0, 14.0, [1.0, 2.0], "r1"], [21, 2, 24, 12.0, 15.0, [2.0, 2.0], "r2"]]
rdd = spark.sparkContext.parallelize(test_rows)
df = spark.createDataFrame(rdd, schema)
df.write.format("tfrecords").option("recordType", "Example").save(path)

df = spark.read.format("tfrecords").option("recordType", "Example").load(path)
df.show()

+-------+---------+---------+----------+--------+----------+---+
|LongCol|DoubleCol|StringCol|IntegerCol|FloatCol| VectorCol| id|
+-------+---------+---------+----------+--------+----------+---+
|     23|     14.0|       r1|         1|    10.0|[1.0, 2.0]| 11|
|     24|     15.0|       r2|         2|    12.0|[2.0, 2.0]| 21|
+-------+---------+---------+----------+--------+----------+---+

